## Generative Adversarial Networks (GANs) - 1 Dimension Data

In [ ]:
# import packages
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

In [ ]:
# discriminator model 

def discriminator(n_inputs=2):

	model = Sequential()
 
	model.add(Dense(50, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
 
  # kernel_initializer='he_uniform' - draws samples from a uniform distribution within [-limit, limit], 
  # where limit = sqrt(6 / fan_in) (fan_in is the number of input units in the weight tensor).

	model.add(Dense(1, activation='sigmoid'))
 
	# the model is complied
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 
	return model

In [ ]:
# generator model

def generator(latent_dim, n_outputs=2):

	model = Sequential()
 
	model.add(Dense(50, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
  
  #kernel_initializer='he_uniform' - draws samples from a uniform distribution within [-limit, limit], 
  # where limit = sqrt(6 / fan_in) (fan_in is the number of input units in the weight tensor).
 
	model.add(Dense(n_outputs, activation='linear'))
 
	return model

In [ ]:
# combine generator and discriminator models, update the generator

def gan(generator_model, discriminator_model):

	# discriminator weights are not trainable
	discriminator.trainable = False

	# connect generator and discriminator
	model = Sequential()
 
	# generator is added
	model.add(generator)
 
	# discriminator is added
	model.add(discriminator)
 
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
 
	return model

In [ ]:
# data generation of n samples with class labels

def real_samples(n):

	# data is generated between -0.5 to 0.5
	input = rand(n) - 0.5

	# generate outputs X^2
	output = input * input

	# stack arrays
	input = input.reshape(n, 1)
	output = output.reshape(n, 1)
 
	stack_arrays = hstack((input, output))
  # numpy.hstack(tup) - stack arrays in sequence horizontally (column wise)
  # This is equivalent to concatenation along the second axis, except for 1-D arrays 
  # where it concatenates along the first axis. Rebuilds arrays divided by hsplit.
 
	# class labels are generated
	class_labels = ones((n, 1))
 
	return stack_arrays, class_labels 

In [ ]:
# latent space is generated and given as input for the generator

def latent_space(latent_dim, n):

	# points in the latent space are generated
	points_latent = randn(latent_dim * n)
 
	# to feed it for the network, reshape into a batch of inputs
	points_latent = points_latent.reshape(n, latent_dim)
 
	return points_latent

In [ ]:
# use the generator to generate n fake examples, with class labels

def fake_samples(generator, latent_dim, n):

	# points are generated in the latent space
	fake_input = latent_space(latent_dim, n)
 
	# predict outputs
	output = generator.predict(fake_input)
 
	# create class labels
	class_labels = zeros((n, 1))
 
	return output, class_labels


In [ ]:
# discriminator is evaluated and the results of real vs fake points are plotted

def performance(epoch, generator, discriminator, latent_dim, n=100):

	# original samples are generated
	original_data, original_label = real_samples(n)
 
	# discriminator performance on original samples are evaluated
	original_loss, original_accuracy = discriminator.evaluate(original_data, original_label, verbose=0)
 
	# fake samples are generated
	fake_data, fake_label = fake_samples(generator, latent_dim, n)
 
	# discriminator performance on fake samples are evaluated
	fake_loss, fake_accuracy = discriminator.evaluate(fake_data, fake_label, verbose=0)
 
	# for every epoch, original accuracy and fake accuracy of discriminator is printed
	print(epoch, original_accuracy, fake_accuracy)
 
	# plot the original and fake points
	pyplot.scatter(original_data[:, 0], original_data[:, 1], color='red')
	pyplot.scatter(fake_data[:, 0], fake_data[:, 1], color='blue')
 
	# every 2000 epochs, the plots are saved to files
	file_name = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(file_name)
	pyplot.close()


In [ ]:
# train the generator and discriminator

def train_generator_discriminator(generator_model, discriminator_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=2000):

	# to update the discriminator, calculate half the size of one batch
	batch_half = int(n_batch / 2)
 
	# enumerate epochs manually 
	for i in range(n_epochs):
   
		# original samples are prepared
		original_data, original_label = real_samples(batch_half)
  
		# fake samples are prepared
		fake_data, fake_label = fake_samples(generator_model, latent_dim, batch_half)
  
		# the discriminator is updated
		discriminator_model.train_on_batch(original_data, original_label)
		discriminator_model.train_on_batch(fake_data, fake_label)
  
		# as input for the generator prepare points in the latent space
		x_gan = latent_space(latent_dim, n_batch)
  
		# for the fake samples create inverted labels 
		y_gan = ones((n_batch, 1))
  
		# using the discriminator's error update the generator
		gan_model.train_on_batch(x_gan, y_gan)
  
		# print the models performance every 2000 epochs
		if (i+1) % n_eval == 0:
			performance(i, generator_model, discriminator_model, latent_dim)

In [ ]:
# latent space size is set to 5
latent_dim = 5

# call the discriminator function
discriminator = discriminator()

# call the generator function
generator = generator(latent_dim)

# call the gan function
gan_model = gan(generator, discriminator)

# train the generator and discriminator model
train_generator_discriminator(generator, discriminator, gan_model, latent_dim)

1999 0.6399999856948853 1.0
3999 0.5899999737739563 0.5400000214576721
5999 0.6399999856948853 0.5400000214576721
7999 0.7400000095367432 0.5600000023841858
9999 0.5400000214576721 0.44999998807907104


Acknowledgement: Jason Brownlee, Generative Adversarial Networks with Python, Machine Learning Mastery, Available from https://machinelearningmastery.com/generative_adversarial_networks/, accessed  March 15th, 2021.